In [ ]:
import duckdb
import polars as pl
# duckdb for 1 month 1 minute stock parquet
# data_dir = "../data/lake/us_stocks_sip/minute_aggs_v1/2025/07/*.parquet"
data_dir = "../data/lake/us_stocks_sip/day_aggs_v1/*/*/*.parquet"
con = duckdb.connect()

result = con.sql(f"""
SELECT
    ticker,
    MAX(high) as max_high,
    MIN(low) as min_low,
    SUM(volume) AS total_volume,
    SUM(volume * close) AS turnover
FROM '{data_dir}'
GROUP BY ticker
HAVING MAX(high) BETWEEN 37.20 AND 37.30
    AND MIN(low) BETWEEN 14.20 AND 14.60
""").pl()
# WHERE ticker = 'TSE'

# 方法1: 设置 Polars 显示选项

with pl.Config(tbl_rows=100, tbl_cols=10, fmt_str_lengths=50):
    print(result)
    
# zstd level 3 653MB 18-20s
# zstd level 10 631MB 18-20s

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

shape: (1, 5)
┌────────┬───────────┬─────────┬───────────────┬──────────┐
│ ticker ┆ max_high  ┆ min_low ┆ total_volume  ┆ turnover │
│ ---    ┆ ---       ┆ ---     ┆ ---           ┆ ---      │
│ str    ┆ f32       ┆ f32     ┆ decimal[38,0] ┆ f64      │
╞════════╪═══════════╪═════════╪═══════════════╪══════════╡
│ QUBX   ┆ 37.240002 ┆ 14.3904 ┆ 7105283       ┆ 1.6908e8 │
└────────┴───────────┴─────────┴───────────────┴──────────┘


In [ ]:
import duckdb
import polars as pl

# splits data read

splits_dir = '../data/raw/us_stocks_sip/splits/splits.parquet'

# 1. polars
# splits = pl.read_parquet(splits_dir)
# result = splits.filter(pl.col('ticker') == 'ENVX')

# 2. duckdb
con = duckdb.connect()

result = con.sql(f"""
                 SELECT * FROM '{splits_dir}'
                 WHERE ticker ='ENVX'
                 """
).pl()

with pl.Config(tbl_rows=20, tbl_cols=20, fmt_str_lengths=100):
    print(result)
    



shape: (1, 5)
┌────────────────────────────────────────────────┬────────────────┬────────────┬──────────┬────────┐
│ id                                             ┆ execution_date ┆ split_from ┆ split_to ┆ ticker │
│ ---                                            ┆ ---            ┆ ---        ┆ ---      ┆ ---    │
│ str                                            ┆ str            ┆ f64        ┆ f64      ┆ str    │
╞════════════════════════════════════════════════╪════════════════╪════════════╪══════════╪════════╡
│ E1a300d5b5f47a368890f04f9508c697580362a81d7411 ┆ 2025-07-17     ┆ 7.0        ┆ 8.0      ┆ ENVX   │
│ 50703db9e488cd95625                            ┆                ┆            ┆          ┆        │
└────────────────────────────────────────────────┴────────────────┴────────────┴──────────┴────────┘
